<a href="https://colab.research.google.com/github/joanne2363/Joanne/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorboardX
# !pip install efficientnet_pytorch

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!mkdir -p models

In [0]:
%%writefile models/__init__.py
from .AlexNet import AlexNet

Writing models/__init__.py


In [0]:
import sys
sys.path.append('models')

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# augmentation = drive.CreateFile({'id':'1qw9CVmePKFtWn5AFoJjalULuTlzTbibO'})
# augmentation.GetContentFile('augmentation.py')
# addataset = drive.CreateFile({'id':'15An-7_wQ1a3QZ41UapjlVewZXTRjl5vt'})
# addataset.GetContentFile('addataset.py')
# admodels = drive.CreateFile({'id':'1dINUuA5HXRP8KxR0FReycZh0mpApNLZS'})
# admodels.GetContentFile('admodels.py')
utils = drive.CreateFile({'id':'1r624iAVyqp_ExFnls7ZgaGjeGzabbYPL'})
utils.GetContentFile('utils.py')
config = drive.CreateFile({'id':'1chG7-jTIDkgaxzyDCCGFdkuAcjYeXojV'})
config.GetContentFile('config.py')

In [0]:
from config import opt
# import torchvision.models as M
import pandas as pd
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models
from efficientnet_pytorch import EfficientNet
import torch.nn as nn
from tensorboardX import SummaryWriter
%load_ext tensorboard
import datetime
import os
import torch
from torch.autograd import Variable
import utils
# from utils import RunningMean
from PIL import Image
import numpy as np
import random
# import models
import time
from utils import AverageMeter
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
torch.backends.cudnn.benchmark = True

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
def main(**kwargs):
  def getmodel(arch,pretrained,num_class):
    # print('[+] loading model...', end = '',flush = True)
    # model = M.resnet50(pretrained=True)
    # model.fc = torch.nn.Linear(2048, 9)
    # print("=> using pre-trained model '{}'".format(model))
    # model.cuda()
    # print('Done')
    # return model
    if arch == 'efficientnet': 
      model = EfficientNet.from_pretrained('efficientnet-b0')
      model._fc.out_features = num_class

    elif arch == 'resnet101':
      model = models.resnet101(pretrained = pretrained)
      num_ftrs = model.fc.in_features
      model.fc = torch.nn.Linear(num_ftrs, num_class)

    elif arch == 'alexnet':
      model = models.alexnet(pretrained = pretrained)
      num_ftrs = model.classifier[6].in_features
      model.classifier[6] = nn.Linear(num_ftrs, num_class)

    elif arch == 'vgg16':
      model = models.vgg16(pretrained = pretrained)
      num_ftrs = model.classifier[6].in_features
      model.classifier[6] = torch.nn.Linear(num_ftrs,num_class)

    elif arch == 'densenet121':
      model = models.densenet121(pretrained = pretrained)
      num_ftrs = model.classifier.in_features
      model.classifier = torch.nn.Linear(num_ftrs, num_class)
    
    elif arch == 'googlenet':
      model = models.googlenet(pretrained = pretrained)
      num_ftrs = model.fc.in_features
      model.fc = torch.nn.Linear(num_ftrs, num_class)
    
    elif arch == 'inception_v3':
      model = models.inception_v3(pretrained = pretrained)
      num_ftrs = model.AuxLogits.fc.in_features
      model.AuxLogits.fc = torch.nn.Linear(num_ftrs, num_class)
      num_ftrs = model.fc.in_features
      model.fc = torch.nn.Linear(num_ftrs, num_class)
    
    print("=> using pre-trained model '{}'".format(model))
    return model


  def train(train_loader, model, criterion, optimizer, epoch, arch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    acc = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    # 從訓練集迭代器中獲取訓練數據
    for i, (images, target) in enumerate(train_loader):
      # 評估圖片讀取耗時
      data_time.update(time.time() - end)
      # 將圖片標籤轉化為tensor
      # image_var = torch.tensor(images).cuda(async=True)
      # label = torch.tensor(target).cuda(async=True)
      image_var = torch.tensor(images).cuda()
      label = torch.tensor(target).cuda()

      # 將圖片輸入網絡，前傳，生成預測值
      y_pred = model(image_var)

      # 計算 loss
      if arch == 'inception_v3':
        loss = criterion(y_pred[0], label)
      else:
        loss = criterion(y_pred, label)
      losses.update(loss.item(), images.size(0))

      # 計算 top1 正確率
      if arch == 'inception_v3':
        prec, PRED_COUNT = utils.accuracy(y_pred[0].data, target, topk=(1, 1))
      else:
        prec, PRED_COUNT = utils.accuracy(y_pred.data, target, topk=(1, 1))
      acc.update(prec, PRED_COUNT)

      # 對梯度進行反向傳播，使用隨機梯度下降更新網絡權重
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      
      # 評估訓練耗時
      batch_time.update(time.time() - end)
      end = time.time()

      # 打印耗时与结果
      if i % print_freq == 0:
        print('Epoch: [{0}][{1}/{2}]\t'
              'Train Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
              'Trans Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
              'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
              'Accuray {acc.val:.3f} ({acc.avg:.3f})'.format(
               epoch, i, len(train_loader), batch_time = batch_time, 
               data_time = data_time, loss = losses, acc = acc)
              )
      
      # 創建 train 日誌文件
      if not os.path.exists(txtlog_path_pre):
        os.makedirs(txtlog_path_pre)
        with open(traintxtlog_path, 'w') as acc_file:
          pass
      # 在日志文件中紀錄每個epoch的精度和loss
      with open(traintxtlog_path, 'a') as acc_file:
        # acc_file.write('Epoch: [{0}][{1}/{2}]\t'
        #                'Train Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
        #                'Trans Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
        #                'Loss {loss.val:.3f} ({loss.avg:.3f})\t'
        #                'Accuray {acc.val:.3f} ({acc.avg:.3f})\t \n'.format(
        #                 epoch, i, len(train_loader), batch_time = batch_time,
        #                 data_time = data_time, loss = losses, acc = acc)
        #               )
        acc_file.write('Epoch:[%2d][%2d/%2d], Train Time(s):%.3f(,%.3f), Trans Time(s):%.3f(,%.3f), Loss:%.3f(,%.3f), Accuray:%.3f(,%.3f) \n '
                       % (epoch, i, len(train_loader),batch_time.val, batch_time.avg,
                          data_time.val, data_time.avg,
                          losses.val, losses.avg,acc.val, acc.avg)
                      )
    return acc.avg, losses.avg


  # 驗證函數
  def validate(val_loader, model, criterion, arch):
    batch_time = AverageMeter()
    losses = AverageMeter()
    acc = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (images, labels) in enumerate(val_loader):
      # image_var = torch.tensor(images).cuda(async=True)
      # target = torch.tensor(labels).cuda(async=True)
      image_var = torch.tensor(images).cuda()
      target = torch.tensor(labels).cuda()

      # 圖片前傳。驗證和測試時不需要更新網絡權重，所以使用 torch.no_grad()，表示不計算權重
      with torch.no_grad():
        y_pred = model(image_var)
        loss = criterion(y_pred, target)
      
      # measure accuracy
      prec, PRED_COUNT = utils.accuracy(y_pred.data, labels, topk=(1, 1))
      losses.update(loss.item(), images.size(0))
      acc.update(prec, PRED_COUNT)

      # measure training time
      batch_time.update(time.time() - end)
      end = time.time()

      if i % print_freq == 0:
        print('TrainVal: [{0}/{1}]\t'
              'Train Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
              'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
              'Accuray {acc.val:.3f} ({acc.avg:.3f})'.format(
            i, len(val_loader), batch_time=batch_time, loss=losses, acc=acc))

    print(' * Accuray {acc.avg:.3f}'.format(acc=acc), '(Previous Best Acc: %.3f)' % best_precision,
          ' * Loss {loss.avg:.3f}'.format(loss=losses), 'Previous Lowest Loss: %.3f)' % lowest_loss)
    return acc.avg, losses.avg


  # 程序主体
  torch.cuda.empty_cache()
  opt.parse(kwargs)
  traindir = opt.train_data_root
  valdir = opt.val_data_root
  print('=========================================')
  date = str(datetime.date.today())
  basic_path = '/content/drive/My Drive/Colab Notebooks/aiad_2/'
  best_path_pre   = basic_path + 'checkpoints/' + date + '/' + opt.arch + '/'
  checkpoint_path = best_path_pre + str(opt.num_class) + '_' + date + '_checkpoint.pth'
  best_loss_path  = best_path_pre + str(opt.num_class) + '_' + date + '_loss_best.pth'
  best_acc_path   = best_path_pre + str(opt.num_class) + '_' + date + '_acc_best.pth'

  txtlog_path_pre  = basic_path + 'logs/' + date + '/' + opt.arch + '/'
  txtlog_path      = txtlog_path_pre + str(opt.num_class) + '_' + 'txtlog.txt'
  traintxtlog_path = txtlog_path_pre + str(opt.num_class) + '_' + 'traintxtlog.txt'
  besttxtlog_path  = txtlog_path_pre + str(opt.num_class) + '_' + 'besttxtlog.txt'
  tensorBoard_path = txtlog_path_pre + 'tensorBoardX/' + str(opt.num_class) + '/'

  # print('traindir:',traindir)
  # print('valdir:',valdir)
  # print('best_loss_path:', best_loss_path)
  # print('best_path_pre:', best_path_pre)
  # print('tensorBoard_path:', tensorBoard_path)
  # print('arch:', opt.arch)
  # print('pretrained:', opt.pretrained)
  # print('num_class:', opt.num_class)
  if not os.path.exists(tensorBoard_path):
        os.makedirs(tensorBoard_path)
  writer = SummaryWriter(tensorBoard_path)

  lr = 1e-1
  best_precision = 0
  lowest_loss = 100

  # 設定打印頻率，即多少step打印一次，用於觀察loss和acc的實時變化
  # 打印結果中，括號前面為實時loss和acc,括號內為epoch內平均loos和acc
  print_freq = 1
  # 驗證集比例
  # val_ratio = 0.12
  # 是否只驗證，不訓練
  evaluate = False
  # 是否從斷點繼續跑
  resume = False
  # 忍受該指標多少個step不變化，就調整學習率
  patience = 0
  # 圖片歸一化，由於採用Pytorch預訓練網絡，因此這裡直接採用Pytorch網絡的參數
  normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

  train_dataset = datasets.ImageFolder(
                    opt.train_data_root,
                    transforms.Compose([
                        # transforms.RandomResizedCrop(IMAGE_SIZE),
                        transforms.Resize(opt.input_size + 32),
                        transforms.CenterCrop(opt.input_size),
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                        normalize,
                    ])
                  )
  val_dataset = datasets.ImageFolder(
                  opt.val_data_root,
                  transforms.Compose([
                      transforms.Resize(opt.input_size + 32),
                      transforms.CenterCrop(opt.input_size),
                      transforms.ToTensor(),
                      normalize,
                  ])
                )
 # 生成圖片迭代器
  train_dataLoader = DataLoader(dataset = train_dataset, batch_size = opt.batch_size, 
                                shuffle = True, pin_memory = True, num_workers = opt.num_workers )
  val_dataLoader = DataLoader(dataset = val_dataset, batch_size = opt.batch_size, 
                              shuffle = False, pin_memory = False, num_workers = 1)
  # 創建模型
  model = getmodel(opt.arch,opt.pretrained,opt.num_class)
  if opt.use_gpu: model.cuda()

  # 使用交叉熵損失函數
  criterion = nn.CrossEntropyLoss().cuda()
  
  # 優化器，使用帶amsgrad的Adam
  optimizer = torch.optim.Adam(model.parameters(), lr, weight_decay = opt.weight_decay, amsgrad = True)
  
  if evaluate:
    validate(val_dataLoader, model, criterion)
  else:
    # 開始訓練
    for epoch in range(opt.max_epoch):
      if patience == 3:
        patience = 0
        model.load_state_dict(torch.load(best_loss_path)['state_dict'])
        lr = lr / 10
        print('loss has increased lr divide 10 lr now is :%e'%(lr))
        # 優化器，使用帶amsgrad的Adam
        optimizer = torch.optim.Adam(model.parameters(), lr, weight_decay = opt.weight_decay, amsgrad = True)

      # train for one epoch
      precisiont, avg_losst = train(train_dataLoader, model, criterion, optimizer, epoch, opt.arch)
      # train(train_dataLoader, model, criterion, optimizer, epoch, opt.arch)
      writer.add_scalar('Train/Acc', precisiont, epoch + 1)
      writer.add_scalar('Train/Loss', avg_losst, epoch + 1)
      # evaluate on validation set
      precision, avg_loss = validate(val_dataLoader, model, criterion, opt.arch)
      writer.add_scalar('Val/Acc', precision, epoch + 1)
      writer.add_scalar('Val/Loss', avg_loss, epoch + 1)

      # print('epoch:',epoch + 1)
      # print('precisiont:',precisiont)
      # print('avg_losst:',avg_losst)
      # print('precision:',precision)
      # print('avg_loss:',avg_loss)

      # 創建日誌文件
      if not os.path.exists(best_path_pre):
        os.makedirs(best_path_pre)
        with open(txtlog_path, 'w') as acc_file:
          pass
      # 在日志文件中紀錄每個epoch的精度和loss
      with open(txtlog_path, 'a') as acc_file:
        acc_file.write('Epoch: %2d, Val Precision: %.4f, Val Loss: %.4f, DateTime: %s \n' 
                       % (epoch, precision, avg_loss, 
                          time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time()))
                         )
                      )
 
      # 紀錄最高精度與最低loss，保存最新模型與最佳模型
      is_best = precision > best_precision
      is_lowest_loss = avg_loss < lowest_loss
      best_precision = max(precision, best_precision)
      lowest_loss = min(avg_loss, lowest_loss)
      state = {
          'epoch': epoch,
          'state_dict': model.state_dict(),
          'best_precision': best_precision,
          'lowest_loss': lowest_loss,
          # 'stage': stage,
          'lr': lr,
      }
      utils.save_checkpoint(state, is_best, is_lowest_loss, checkpoint_path, best_acc_path, best_loss_path)
      
      if not is_best:
        patience += 1
      # # 判斷是否進行下一個stage
      # if (epoch + 1) in np.cumsum(stage_epochs)[:-1]:
      #   stage += 1
      #   optimizer = adjust_learning_rate()
      #   model.load_state_dict(torch.load(BEST_ACC_PATH)['state_dict'])
      #   print('Step into next stage')
      #   with open(TXTLOG_PATH, 'a') as acc_file:
      #     acc_file.write('---------------Step into next stage----------------\n')
  
  # 紀錄最佳分數
  if not os.path.exists(best_path_pre):
    os.makedirs(best_path_pre)
    with open(txtlog_path, 'w') as acc_file:
      pass
  with open(txtlog_path, 'a') as acc_file:
    acc_file.write('%s  * best acc: %.8f  \n' 
                    % (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time())),best_precision)
                  )


In [0]:
main(max_epoch = 60, arch = 'googlenet', num_class = 16, evaluate = False, batch_size = 64, input_size = 299,
     train_data_root = '/content/drive/My Drive/Colab Notebooks/aiad_2/data/train_16/',
     val_data_root = '/content/drive/My Drive/Colab Notebooks/aiad_2/data/val_16/' )

user config:
env default
arch googlenet
pretrained True
num_class 16
input_size 299
evaluate False
train_data_root /content/drive/My Drive/Colab Notebooks/aiad_2/data/train_16/
val_data_root /content/drive/My Drive/Colab Notebooks/aiad_2/data/val_16/
load_model_path /content/drive/My Drive/Colab Notebooks/aiad_2/checkpoints/model.pth
batch_size 64
use_gpu True
num_workers 4
print_freq 20
debug_file /tmp/debug
result_file result.csv
max_epoch 60
lr 0.1
lr_decay 0.95
weight_decay 0.0001
img_size 224
parse <bound method parse of <config.DefaultConfig object at 0x7f4c56fdda90>>


Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:02<00:00, 19.4MB/s]

=> using pre-trained model 'GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, mome

KeyboardInterrupt: ignored

In [0]:
main(max_epoch = 40,arch = 'alexnet', num_class = 16, evaluate = False, batch_size = 64,
     train_data_root = '/content/drive/My Drive/Colab Notebooks/aiad_2/data/train_16/',
     val_data_root = '/content/drive/My Drive/Colab Notebooks/aiad_2/data/val_16/' )

In [0]:
%load_ext tensorboard
%tensorboard --logdir '/content/drive/My Drive/Colab Notebooks/aiad_2/logs/'

In [0]:
import torchvision.models as models
import torch
model = models.inception_v3(pretrained = True)
print("=> using pre-trained model '{}'".format(model))